# テキストマイニング: Twitter

## 1. はじめに<a class="anchor" id="intro-bullet"></a>
### 1.1. 目的
### 1.2. 目次

1. **[はじめに](#intro-bullet)**
2. **[データの前処理](#first-bullet)**
3. **[形態素解析; Mecab](#second-bullet)**
4. **[テキストの出現頻度の可視化; Wordcloud](#third-bullet)**
5. **[テキストの繋がり; 共起ネットワーク](#fourth-bullet)**
6. **[おわりに](#final-bullet)**

## 2. データの前処理<a class="anchor" id="first-bullet"></a>

## 3. 形態素解析; Mecab<a class="anchor" id="second-bullet"></a>

## 4. テキストの出現頻度の可視化; Wordcloud<a class="anchor" id="third-bullet"></a>

## 5. テキストの繋がり; 共起ネットワーク<a class="anchor" id="fourth-bullet"></a>

## 6. おわりに<a class="anchor" id="final-bullet"></a>